# 신경망과의 세 번째 만남
by uramoon@kw.ac.kr (<a href="https://raw.githubusercontent.com/ronreiter/interactive-tutorials/master/LICENSE">Apache 2.0 License</a>)

2004년 MS 연구소에서 발표한 논문에 포함된 MLP (Multi-Layer Perceptron, 유닛 800개 들어간 은닉층 하나만 사용)의 MNIST 테스트 정확도는 98.4%입니다. 같은 논문에서 훈련 데이터를 늘리고 이미지를 이차원 형태로 해석하는 능력을 갖춘 합성곱 신경망 (CNN, Convolution neural network)을 사용했을 때 99.6%를 달성했습니다.



본 노트북에서는 MLP의 epochs를 자동으로 설정하여 테스트 정확도를 98.3% 이상으로 만드는 것을 목표로 합니다.

In [1]:
import keras
keras.__version__

'2.11.0'

## 데이터셋 불러오기 & 이미지 전처리


In [2]:
# TODO: MNIST 데이터셋 불러오기
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# TODO: 이미지 전처리
# [0, 255]의 3차원 배열을 [0, 1]의 2차원 배열로 변환
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255.0
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## 분류 문제의 출력층과 손실 (loss) 함수

출력 유닛의 수가 **2 이상**인 분류 문제에서 출력층의 활성화 함수로는 지난 시간에 사용한 **softmax**를 사용합니다. 출력층의 각 유닛은 일반적으로 $(-\infty,+\infty)$의 값을 출력하는데 softmax는 그들 중 가장 큰 값에 가장 높은 확률을 부여합니다. (동시에 모든 확률을 더했을 때 1이 되도록 만듦)


손실 (loss) 함수는 출력층의 출력 내용이 이상적인 출력 내용과 얼마나 동떨어졌는지 계산합니다. (loss가 클수록 나쁨) 

출력 유닛의 수가 **2 이상**인 분류 문제에서 사용할 수 있는 대표적인 손실 함수는 다음의 두 개가 있습니다:

1. categorical_crossentropy: 지난 시간에 사용한 함수로 정답 (label)이 5인 경우  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]와 같이 **label을 변환**해야 한다.
2. sparse_categorical_crossentropy: 정답 (label)이 5인 경우 5를 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]로 해석하여 loss를 계산하므로 **label을 변환할 필요가 없다.** (5를 봤을 때 5번째 인덱스의 출력은 1이고, 나머지는 0으로 해석)

이 번 노트북에서는 **label을 변환하지 않고, sparse_categorical_crossentropy를 사용해봅시다.**

--- 

출력 유닛의 수가 1인 이진 분류 (binary classification)의 출력층에서는 **softmax를 사용하면 안됩니다.** softmax는 여러 개의 출력을 확률로 변환하는데, 출력이 하나 밖에 없으므로 하나의 출력을 무조건 1로 만들어주기 때문입니다. 이진 분류 문제에서는 출력층에서 sigmoid 함수를 사용하여 $(-\infty,+\infty)$의 값을 (0, 1)의 값으로 바꿔줍니다. (예: 고양이와 개를 분류할 때 고양이일 확률이 $p$라면, 개일 확률은 $1-p$로 해석합니다.)



## MLP 만들기

문제에 따라서 효율적인 인공신경망 구조가 있으나 직접 해보기 전까지는 어떠한 구조가 잘 동작할 지 알 수 없습니다.<br> 편의상 신경망과의 첫 만남 때 사용한 것과 동일한 네트워크를 사용합니다.

In [5]:
# TODO: 네트워크 만들기, loss만 써주세요.
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Input(784,)) # (784)는 784라는 정수, (784,)는 길이 784인 일차원 배열을 의미
network.add(layers.Dense(512, activation='relu')) 
network.add(layers.Dense(10, activation='softmax')) 

network.compile(optimizer='rmsprop',              
                loss='sparse_categorical_crossentropy',  
                metrics=['accuracy'])          

## EarlyStopping 사용하기

epochs를 무작정 키울 수 없는 것은 인공신경망이 훈련 데이터에 과적합되어 보지 못한 테스트 데이터에서의 성능이 떨어질까 걱정되기 때문입니다. 이를 방지하려면 훈련 데이터의 일부를 훈련 중 볼 수 없는 **검증 데이터**로 활용하면 됩니다.

**검증 데이터**를 사용하면 테스트 데이터를 사용하지 않고도 훈련 중에 **테스트 데이터에서의 성능을 미리 가늠**해볼 수 있습니다. 훈련 중 epoch이 끝날 때마다 검증 데이터에서의 성능을 평가하여 과적합 조짐 (훈련 데이터에서의 성능은 올라갔으나 훈련 중 보지 못한 검증 데이터에서의 성능은 하락)이 보이면 훈련을 그만두도록 설정할 수 있습니다.

In [7]:
# TODO: EarlyStopping을 사용하여 훈련하기
# Test Accuracy 98.3% 이상을 달성하세요.
# 처음부터 훈련하려면 위의 코드블록도 다시 실행하고, 기존의 신경망으로 이어서 훈련하려면 이 코드 블록만 다시 실행합니다.
from keras.callbacks import EarlyStopping

network.fit(train_images, train_labels, batch_size=128, epochs=10000, # 수정불가: 만 번의 epochs 전에 조기종료를 할 것입니다.
            callbacks=EarlyStopping(patience=7),                  # 검증 데이터에서의 val_loss 성능 하락을 최대 몇 번 참을지 적절하게 설정
            validation_split=0.1)                                     # 훈련 데이터에서 검증 데이터로 얼마나 사용할지 적절하게 설정

test_loss, test_acc = network.evaluate(test_images, test_labels)
print(f'Test Accuracy:{test_acc * 100:.2f}%')

Epoch 1/10000
422/422 [==============================] - 1s 3ms/step - loss: 0.0097 - accuracy: 0.9978 - val_loss: 0.0709 - val_accuracy: 0.9808
Epoch 2/10000
422/422 [==============================] - 1s 3ms/step - loss: 0.0073 - accuracy: 0.9985 - val_loss: 0.0638 - val_accuracy: 0.9843
Epoch 3/10000
422/422 [==============================] - 1s 3ms/step - loss: 0.0055 - accuracy: 0.9990 - val_loss: 0.0634 - val_accuracy: 0.9838
Epoch 4/10000
422/422 [==============================] - 1s 3ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 0.0676 - val_accuracy: 0.9847
Epoch 5/10000
422/422 [==============================] - 2s 4ms/step - loss: 0.0026 - accuracy: 0.9996 - val_loss: 0.0678 - val_accuracy: 0.9843
Epoch 6/10000
422/422 [==============================] - 1s 3ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.0746 - val_accuracy: 0.9835
Epoch 7/10000
422/422 [==============================] - 1s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0655 - val_ac

epoch이 진행되며 훈련 데이터에서의 성능은 비교적 꾸준히 좋아지지만 (loss는 낮아지고, accuracy는 높아짐) 검증 데이터로 측정한 val_loss와 val_accuracy는 그렇지 않음을 확인해보세요.

---

MNIST에서 MLP 사용 시 규제가 크게 효과적이지 않아 검증 데이터 없이 모든 훈련 데이터에 과적합시키면 테스트 성능이 좋게 나오는 경향이 있습니다. 

일반적으로는 전체 데이터에서 훈련:검증:테스트의 비율을 6:2:2로 설정하거나 MNIST에서와 같이 훈련 데이터와 테스트 데이터가 따로 제공되는 경우에는 훈련 데이터의 20% 정도를 검증 데이터로 사용합니다. (MNIST에서는 20%로 설정하면 좋지 않음)

앞으로는 얼마나 많은 epochs을 훈련해야 하는지 감이 없을 때 검증 데이터와 EarlyStopping을 활용해 봅시다. (다만 훈련 데이터가 충분하지 않은 경우에는 모두 훈련 데이터로 사용합니다.)

## 모델 저장하기

1. save 함수 이용 (예: network.save('my_mnist.h5')), 보통 확장자는 h5 사용
2. 왼쪽 메뉴에서 폴더 모양의 아이콘 (파일) 클릭
3. 방금 저장한 파일을 로컬로 다운로드 하기 (점 세개 클릭하면 다운로드 가능, 파일이 안보이면 새로고침 클릭)
4. 다운로드 받은 파일 용량 확인해보기

다운로드한 모델도 제출합니다.

In [8]:
network.save('my_mnist.h5')